# Deep Learning with GPU (CPU)

Before starting the course http://course.fast.ai/, we need to configure libraries to work with GPU. Set up free environment (not like Crestle or Paperspace).


# Install and configure Theano

~~pip install theano~~<br>
conda install theano pygpu

echo "[global]<br>
device = cpu<br>
floatX = float32<br>
[cuda]<br>
root = /usr/local/cuda" > ~/.theanorc

# Testing Theano

Testing Theano with GPU:<br>
http://deeplearning.net/software/theano/tutorial/using_gpu.html

By default, when device indicates preference for GPU computations, Theano will **fall back to the CPU** if there is a problem with the GPU. You can use the flag force_device=True to instead raise an error when Theano cannot use the GPU.

In [1]:
from theano import function, config, shared, tensor
import numpy
import time

vlen = 10 * 30 * 768  # 10 x #cores x # threads per core
iters = 1000

rng = numpy.random.RandomState(22)
x = shared(numpy.asarray(rng.rand(vlen), config.floatX))
f = function([], tensor.exp(x))
print(f.maker.fgraph.toposort())
t0 = time.time()
for i in range(iters):
    r = f()
t1 = time.time()
print("Looping %d times took %f seconds" % (iters, t1 - t0))
print("Result is %s" % (r,))
if numpy.any([isinstance(x.op, tensor.Elemwise) and
              ('Gpu' not in type(x.op).__name__)
              for x in f.maker.fgraph.toposort()]):
    print('Used the cpu')
else:
    print('Used the gpu')

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


[Elemwise{exp,no_inplace}(<TensorType(float32, vector)>)]
Looping 1000 times took 78.366021 seconds
Result is [1.2317803 1.6187934 1.5227807 ... 2.2077181 2.2996776 1.6232328]
Used the cpu
